## Self study 6

In this self study we directly continue with what we started in self study 5

**Task 2:** For the same user, and the same train/test split, implement the user-based neighborhood method. How does the RMSE compare to what you got in the content-based approach?

In [217]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

In [218]:
#Load data
R=np.genfromtxt('u.data',dtype=int)

#Sort users per number of ratings
Nratings=np.zeros(943)
for i in range(943):
    Nratings[i]=len(np.where(R[:,0]==i+1)[0])

#Get users with most ratings and make train/test split
users = np.where(Nratings>500)
ratID = np.where(R[::,0] == users[0][0]+1)
ratings = [R[i, 1:3] for i in ratID[0]]

split = len(ratings) // 3
train = pd.DataFrame(ratings[:2*split], columns=["Movie ID", "Rating"])
test = pd.DataFrame(ratings[2*split:], columns=["Movie ID", "Rating"])



In [219]:
def estimate_rating(train, movie, user):
    users = np.where(R[::,1] == movie)
    avg_rating = np.average(train["Rating"])
    train["Relative Rating"] = train["Rating"] - avg_rating
    result = []
    users = [R[i,0] for i in users[0] if i != user]
    for us in users:
        ratIDs = np.where(R[::,0] == us)[0]
        ratings = np.asarray([R[i,1:3] for i in ratIDs])
        
        user_avg = np.average(ratings[::,1])
        ratings = np.c_[ratings, [row[1] - user_avg for row in ratings]]
        common_movies = [int(i) for i in ratings[::,0] if i in train["Movie ID"].values]
        common_movies.sort()
        old_user_ratings = np.asarray([train.values[[np.where(train["Movie ID"].values == mov)[0]], 2][0] for mov in common_movies])[::,0]
        new_user_ratings = np.asarray([ratings[np.where(ratings[::,0] == mov)[0], 2] for mov in common_movies])[::,0]
        sim = np.dot(old_user_ratings, new_user_ratings)/ (np.linalg.norm(old_user_ratings) * np.linalg.norm(new_user_ratings))
        result.append(ratings[np.where(ratings[::,0] == movie)[0][0],1] * sim)
    return np.average(result) + avg_rating




In [220]:
estimate_rating(train, test.values[0,0], users[0][0]+1)


3.9077643022526676

In [223]:
rats = []
for m in test.values:
    r = estimate_rating(train, m[0], users[0][0]+1)
    print(r)
    rats.append(r)
rats = np.asarray(rats)

truth = np.asarray(test['Rating'])
RMSE = math.sqrt(np.average((truth-rats) ** 2))
RMSE


3.9077643022526676
3.774691324485744
3.8639875760766134
3.8649223248949798
4.074859743560783
4.000175511108554
3.682458982968191
3.776770273240651
3.922232709139133
3.802302637873435
3.8617964220364343
3.8967464041495927
3.887686928246365
3.9318470051583625
3.842330252868388
3.803166115001309
3.718930235440606
3.710302364690555
3.852738863402953
3.589608335147553
3.4465262134247725
3.6791194988321188
3.698268715365163
3.9063685677886286
3.966762861977749
3.855110975291473
3.6560722045378187
3.880180877258538
3.94947181871142
3.805235987709871
4.00456192682166
3.931977357465313
3.634144846206902
3.699052521942159
3.6898066116506896
3.7754597269720036
4.005506011946936
3.8185288255334835
3.766094418514344
3.840047739806011
3.7205129493638096
4.042206567915512
3.6999416837116703
3.882490576706231
3.8088476689971946
3.752623039292338
3.980116415588862
3.9828704952239837
3.6751838256777765
3.821520428919476
3.772261272648136
3.490257692493645
3.9864931239441592
3.9608334393449693
3.53428074

1.5238305447405054

**Task 3:** Now take the whole rating matrix. Divide all the ratings in the matrix into a train and a test set. For the user you have used in Tasks 1 and 2, you should include the same ratings in the test set as before. Be sure that you do the train/test division uniformly at random over all ratings, so that you don't get all the ratings of one movie (or one user) included in the test set (that would lead to some cold start problems).  You can also start with a sub-matrix of the whole rating matrix by selecting only a subset of users, and a subset of movies.

Implement the gradient-descent approach for minimizing the error function for the ratings in the training set (you have to pick a value for the number of latent dimensions). Compare the RMSE values you obtain for the test user of tasks 1 and 2 with what you have previously obtained.  